In [18]:
import pandas as pd
import numpy as np
import sys
from matplotlib import colors, cm, pyplot as plt

In [2]:
precursor_id = 1000

In [3]:
before_df = pd.read_csv('/Users/darylwilding-mcbride/Downloads/ms1-before-intensity-descent-precursor-{}.csv'.format(precursor_id))
after_df = pd.read_csv('/Users/darylwilding-mcbride/Downloads/ms1-after-intensity-descent-precursor-{}.csv'.format(precursor_id))

In [6]:
before_df.head()

,mz,intensity
0,606.683332,76.0
1,606.810812,59.0
2,606.830128,37.0
3,607.297675,52.0
4,607.316999,91.0


In [7]:
ms1_df = pd.read_pickle('/Users/darylwilding-mcbride/Downloads/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-precursor-1000-of-52464/exp-P3856-run-P3856_YHE211_1_Slot1-1_1_5104-precursor-1000-cuboid-points-ms1-df.pkl')

In [8]:
ms1_df.head()

,frame_id,frame_type,mz,scan,intensity,retention_time_secs
0,3993,0,605.725731,861,55,447.157516
1,3993,0,605.745030,824,76,447.157516
2,3993,0,605.802929,839,85,447.157516
3,3993,0,605.806789,847,78,447.157516
4,3993,0,605.810649,816,78,447.157516


In [91]:
# create the colour mapping
colour_map = plt.get_cmap('rainbow')
norm = colors.LogNorm(vmin=ms1_df.intensity.min(), vmax=ms1_df.intensity.max(), clip=True)
# norm = colors.Normalize(vmin=ms1_df.intensity.min(), vmax=ms1_df.intensity.max())

In [92]:
ms1_df.intensity.max()

374

In [93]:
# calculate the colour to represent the intensity
colours_l = []
for i in ms1_df.intensity.unique():
    marker_size = (i / ms1_df.intensity.min()) * 2
    colours_l.append((i, colour_map(norm(i), bytes=True)[:3], marker_size))
colours_df = pd.DataFrame(colours_l, columns=['intensity','colour','marker_size'])
ms1_colour_df = pd.merge(ms1_df, colours_df, how='left', left_on=['intensity'], right_on=['intensity'])

In [94]:
ms1_colour_df.sample(n=4)

,frame_id,frame_type,mz,scan,intensity,retention_time_secs,colour,marker_size
1485,4088,0,607.780867,858,59,457.138874,"(130, 254, 178)",13.111111
135,3998,0,608.790388,843,96,447.682742,"(196, 232, 138)",21.333333
2891,4169,0,607.796332,837,72,465.695343,"(156, 250, 163)",16.000000
3138,4184,0,607.792465,829,91,467.269576,"(188, 237, 143)",20.222222


In [95]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

print(__version__) # requires version >= 1.9.0

4.8.1


In [96]:
init_notebook_mode(connected=True)

In [97]:
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]

In [98]:
tableau20_py = tableau20.copy()

In [135]:
import chart_studio.plotly as py
import plotly.graph_objs as go

traces = []

trace = go.Scatter3d(
    x = ms1_df.mz,
    y = ms1_df.retention_time_secs,
    z = ms1_df.scan,
    mode = 'markers',
    marker = dict(
        line = None,
        size = ms1_colour_df.marker_size,
        color = ms1_colour_df.colour,
        opacity = 0.4
    ),
    text = 'hover',
    hoverinfo = 'text',
    name = "MS1"
)
traces.append((trace))

data = traces
layout = go.Layout(
    autosize=False,
    width=1000,
    height=1000,
    title="Precursor cuboid raw data",
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=4
    ),
    scene = dict(
        xaxis = dict(
            title='m/z', autorange='reversed'),
        yaxis = dict(
            title='retention time', autorange='reversed'),
        zaxis = dict(
            title='scan'),
        aspectmode = "manual",
        aspectratio = dict( x = 10, y = 15, z = 5),
        camera = dict(
            up=dict(x=0, y=0, z=1),
            center=dict(x=0, y=0, z=0),
            eye=dict(x=8, y=20, z=8)
        )
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)